In [ ]:
import numpy as np
import astropy.cosmology as cosmology
cosmo = cosmology.FlatLambdaCDM(H0=70, Om0=0.3)
from scipy.stats import norm
import emcee
from datetime import datetime

In [ ]:
num_SNe = 10
types = ['Ia', 'Ibc', 'II']
redshift_bins = np.linspace(0.01, 1.201, num=100, endpoint=True)
distancemodulus_bins = cosmo.distmod(redshift_bins)
theta = [72] #H0
mcrandstep = 10.**-1.
nthreads = 1
nwalkers = 150
nsteps = 2000
ninit = 200
output_chain = True
emcee_chain_output = "emcee_chain_testing.dat"

# ln Joint Interim Posterior (equation 10 of paper)

In [ ]:
def lnprob(theta):
    return lnprior(theta) + np.sum(np.sum(np.sum(np.sum(
                        lninterimposterior + lnhyperlikelihood(theta) -lninterimhyperlikelihood
                                                        , axis=3), axis=2), axis=1), axis=0)

# Priors on cosmological parameters and host galaxy redshift parameters

In [ ]:
def lnprior(theta):
    H0 = theta[0]
    H0prior = norm.logpdf(H0, loc=70, scale=2)
    return H0prior

In [ ]:
def lnhyperlikelihood(theta):
    return 1.0

# MCMC Sampler

In [ ]:
def run_MCMC(theta, lnprob):
    init_positions = [np.array(theta) + mcrandstep*np.random.randn(len(theta)) for i in range(int(nwalkers))]
    sampler = emcee.EnsembleSampler(int(nwalkers), len(theta), lnprob, threads=int(nthreads))
    for i, result in enumerate(sampler.sample(init_positions, iterations=ninit, storechain=False)):
        position = result[0]
        lnprob_pos = result[1]
        if (i+1) % 20 == 0:
            print("Burn-in: {0:.1f}%, ".format(100 * float(i+1) / ninit))
            print datetime.now()
        if output_chain:
            if i%10 == 0:
                f = open(emcee_chain_output, "a")
                for k in range(position.shape[0]):
                    for j in range(position.shape[1]):
                        f.write("{:+010.5f} ".format(position[k,j]))
                    f.write("{:+010.5f} ".format(lnprob_pos[k]))
                    f.write("\n")
                f.close()

In [ ]:
lninterimposterior = np.zeros((num_SNe, len(types), len(redshift_bins), len(distancemodulus_bins)))

In [ ]:
interimtheta = [72]
lninterimhyperlikelihood = lnhyperlikelihood(interimtheta)

In [ ]:
run_MCMC(theta, lnprob)